<a href="https://colab.research.google.com/github/elizabethavargas/Dataset-Description-Generation/blob/main/test_autoddg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/VIDA-NYU/AutoDDG@main

import pandas as pd
from openai import OpenAI

from autoddg import AutoDDG, GPTEvaluator
from autoddg.utils import get_sample

  Cloning https://github.com/VIDA-NYU/AutoDDG (to revision main) to /tmp/pip-req-build-uzzfpxts
  Running command git clone --filter=blob:none --quiet https://github.com/VIDA-NYU/AutoDDG /tmp/pip-req-build-uzzfpxts
  Resolved https://github.com/VIDA-NYU/AutoDDG to commit d70544301a6eba9463a759c7071c463efcba8e8e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.6/460.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 767.5/767.5 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 44.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 60.4 MB/s eta 0:

In [ ]:
# Setup OpenAI client
client = OpenAI(api_key=API_KEY)
# Initialize AutoDDG
autoddg = AutoDDG(client=client, model_name="gpt-5-nano", description_temperature=1.0, )

# Generate description from a small CSV sample
sample_csv = """Case_ID,Age,BMI
C3L-00004,72,22.8
C3L-00010,30,34.15
"""

prompt, description = autoddg.describe_dataset(dataset_sample=sample_csv)

print(description)

In [8]:
prompt

'Answer the question using the following information.\n\nFirst, consider the dataset sample:\n\nCase_ID,Age,BMI\nC3L-00004,72,22.8\nC3L-00010,30,34.15\n\n\nQuestion: Based on the information above and the requirements, provide a dataset description in sentences. Use only natural, readable sentences without special formatting.\n\nAnswer:\n\nTarget length: approximately 100 words.'

In [14]:
# Instantiate AutoDDG
auto_ddg = AutoDDG(client=client, model_name="gpt-5-nano", description_temperature=1.0, topic_temperature=1.0)

# Load dataset
csv_file = "clark_dataset.csv"
title = "Renal Cell Carcinoma"
original_description = (
    "This study reports a large-scale proteogenomic analysis of ccRCC to discern the functional impact "
    "of genomic alterations and provides evidence for rational treatment selection stemming from ccRCC pathobiology"
)
csv_df = pd.read_csv(csv_file)

# Sample rows
sample_df, dataset_sample = get_sample(csv_df, sample_size=100)

# Generate profiles
basic_profile, structural_profile = auto_ddg.profile_dataframe(csv_df)
semantic_profile_details = auto_ddg.analyze_semantics(sample_df)
semantic_profile = "\n".join(
    section for section in [structural_profile, semantic_profile_details] if section
)

# Generate topic
data_topic = auto_ddg.generate_topic(
    title=title,
    original_description=original_description,
    dataset_sample=dataset_sample,
)


/usr/local/lib/python3.12/dist-packages/datamart_profiler/core.py:199: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data = data.astype(object).fillna('').astype(str)


In [16]:
auto_ddg.describe_dataset(data_topic=data_topic, use_profile=True,
                          semantic_profile=semantic_profile, use_topic=True, dataset_sample=sample_csv)

('Answer the question using the following information.\n\nFirst, consider the dataset sample:\n\nCase_ID,Age,BMI\nC3L-00004,72,22.8\nC3L-00010,30,34.15\n\n\nMoreover, the dataset topic is: ccRCC Proteogenomics. Based on this topic, please add sentence(s) describing what this dataset can be used for.\n\nQuestion: Based on the information above and the requirements, provide a dataset description in sentences. Use only natural, readable sentences without special formatting.\n\nAnswer:\n\nTarget length: approximately 100 words.',
 'The dataset focuses on ccRCC Proteogenomics and provides anonymized patient identifiers (Case_ID) along with basic clinical attributes (Age, BMI). The sample includes two records: C3L-00004, Age 72, BMI 22.8; C3L-00010, Age 30, BMI 34.15. This dataset can be used for basic demographic profiling of ccRCC patient cohorts, such as estimating age distribution and BMI ranges. It also serves as a lightweight link to more detailed proteogenomic data, enabling prelimina

In [15]:
data_topic

'ccRCC Proteogenomics'